### Recurent Neural Network for text generation (character level)

Adapted from: https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb

In [1]:
#import unidecode
import string
import random
import re
import torch
import torch.nn as nn
from torch.autograd import Variable

#### Import the corpus - Compilation of Shakespeare texts

In [60]:
# All characters available
all_characters = string.printable
vocab = set(all_characters)
n_characters = len(vocab)

# give every character an index
word_to_ix = {word: i for i, word in enumerate(vocab)}

file = open('./shakespeare.txt').read()
file_len = len(file)
print('The file length is: ', file_len)
print()
print("All existing characters in the corpus: \n"+str(all_characters))
print("Number of unique characters: "+str(n_characters))

The file length is:  1115393

All existing characters in the corpus: 
0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

Number of unique characters: 100


#### Helper function to create chunks of text

In [83]:
def random_chunk(chunk_len):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk(200))

set on. This paltering
Becomes not Rome, nor has Coriolanus
Deserved this so dishonour'd rub, laid falsely
I' the plain way of his merit.

CORIOLANUS:
Tell me of corn!
This was my speech, and I will sp


#### Inputs

In [61]:
def char_tensor(string, word_to_ix):    
    idxs = [word_to_ix[c] for c in string]
    return torch.LongTensor(idxs)

In [62]:
char_tensor('abcDEF', word_to_ix)

tensor([ 88,  11,  32,  13,  94,  42])

#### Targets

The input will be all characters up to the last, and the target will be all characters from the first. <br>
Example: Input:"We shall go!" --> Target: "e shall go!"

In [89]:
def random_training_set(chunk_len):    
    chunk = random_chunk(chunk_len)
    inp = char_tensor(chunk[:-1], word_to_ix)
    target = char_tensor(chunk[1:], word_to_ix)
    return inp, target

In [90]:
random_training_set(10)

(tensor([ 56,  63,  56,  99,  78,  77,   0,  56,  65,  60]),
 tensor([ 63,  56,  99,  78,  77,   0,  56,  65,  60,  56]))

#### Model

In [4]:
class RNN(nn.Module):
    def __init__(self, num_features, num_hidden, num_hidden_layers, num_output):
        super(RNN, self).__init__()
        self.num_hidden = num_hidden
        self.num_hidden_layers = num_hidden_layers
        
        self.rnn = nn.RNN(num_features, num_hidden, num_hidden_layers, 
                          batch_first=True, nonlinearity="relu")
        
        self.linear_out = nn.Linear(num_hidden, num_output)
    
    def forward(self, inputs, hidden):
        output, hidden = self.rnn(inputs, hidden)
        seq_len, batch, input_size
        output = self.linear_out(output)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(self.num_hidden_layers, 1, self.num_hidden)

#### Setup training

In [ ]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len